# LangChain Output Parsers with OpenAI

This notebook demonstrates how to use **LangChain output parsers**
to transform raw LLM responses into structured Python-friendly formats.

The focus is on:
- Parsing raw text output
- Enforcing structured responses via format instructions
- Using built-in output parsers for lists and dates
- Improving reliability of downstream processing

Output parsers are critical when integrating LLMs into
deterministic application pipelines.


In [ ]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser,CommaSeparatedListOutputParser
from datetime import datetime

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Model Initialization

A chat-based OpenAI model is initialized with deterministic
generation settings to ensure consistent and parseable outputs.


In [ ]:
chat = ChatOpenAI(
    model="gpt-5-nano", 
    temperature=0, 
    model_kwargs= {"text":{"verbosity": 'low'},"reasoning":{"effort": "medium"}},
    
    ) 

## Parsing Plain Text Output

The simplest output parser is `StrOutputParser`,
which converts model responses into clean Python strings.

This is useful when downstream logic expects
a normalized text response.


In [ ]:
message_h = HumanMessage(content = "Can you give me an interesting fact I probably don't know about?")

In [ ]:
response = chat.invoke([message_h])
response

In [ ]:
str_output_parser = StrOutputParser()
resposnse_parsed = str_output_parser.invoke(response.text)
resposnse_parsed

## Enforcing List Output with CommaSeparatedListOutputParser

List-based outputs are common in NLP tasks such as:
- Name suggestions
- Keyword extraction
- Feature enumeration

The `CommaSeparatedListOutputParser` enforces
a predictable list format.


In [ ]:
message_h =  HumanMessage(f''' I've recently adoped a dog, Could you suggest some dog names? 

{CommaSeparatedListOutputParser().get_format_instructions()}
''')
print(message_h.content)

In [ ]:
response = chat.invoke([message_h])
print(response.text)

In [ ]:
list_output_parser = CommaSeparatedListOutputParser()
response_parsed = list_output_parser.invoke(response)
response_parsed

In [ ]:
print(list_output_parser.get_format_instructions())

## Parsing Date-Time Outputs

When exact date formats are required,
models can be instructed to return responses
in a strict timestamp format.

This example demonstrates enforcing
an ISO-like datetime string.


In [48]:
DATE_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"

In [ ]:
#Date Parser Example
message_h = HumanMessage(f''' When was the Danish Poet Piet Hiein Born? 
 Return ONLY a datetime string in this exact format:{DATE_FORMAT} 
''')
print(message_h.content)

In [ ]:
response = chat.invoke([message_h])


In [ ]:
print(response.text)

## Summary

This notebook demonstrated:

- Parsing raw LLM output into Python strings  
- Enforcing list outputs using format instructions  
- Converting comma-separated text into structured lists  
- Requesting and validating strict date-time formats  

Output parsers are essential for building
robust, production-grade LLM applications.
